In [2]:
import pandas as pd
import scipy.stats
import numpy as np

In [3]:
board = pd.read_csv("board.csv")

In [53]:
board[board["Position"] == "rb"].head(20)

,pid,Player,Season,AVG,FPTS_TG_pred,STD,Position,VOR
1,16393,Christian McCaffrey SF (9),2023,1.7,17.467539,0.577350,rb,8.751995
3,16483,Austin Ekeler LAC (5),2023,4.0,16.885168,0.000000,rb,8.169624
7,17240,Saquon Barkley NYG (13),2023,8.3,12.037792,1.527525,rb,3.322247
8,23133,Bijan Robinson ATL (11),2023,8.7,15.537254,0.577350,rb,6.821709
9,17246,Nick Chubb CLE (5),2023,11.3,16.411016,1.527525,rb,7.695471
13,19217,Jonathan Taylor IND (11),2023,13.7,14.924777,5.686241,rb,6.209233
16,15514,Derrick Henry TEN (7),2023,17.0,16.199790,4.000000,rb,7.484246
17,18705,Tony Pollard DAL (7),2023,18.3,11.258382,4.509250,rb,2.542837
21,18269,Josh Jacobs LV (13),2023,21.7,13.950144,3.785939,rb,5.234600
24,22726,Rhamondre Stevenson NE (11),2023,24.3,10.634467,1.154701,rb,1.918922


In [5]:
from IPython.display import display, HTML, clear_output


In [6]:
aux_rep = board[board.AVG < 100].sort_values("AVG", ascending=False)

In [7]:
replacement_players = {}
for i, player in aux_rep.iterrows():
    
    pos = player["Position"]
    if pos not in replacement_players.keys():
        replacement_players[pos] = player["FPTS_TG_pred"]
    
    if len(replacement_players.keys()) == 4:
        break
    #rep_df[]

In [8]:
vors = []
for i, player in board.iterrows():
    pos = player["Position"]
    
    avg_pos = replacement_players[pos]
    
    vor = player["FPTS_TG_pred"] - avg_pos
    
    vors.append(vor)

In [9]:
board["VOR"] = vors

In [10]:
replacement_players

{'wr': 8.131173760749355,
 'rb': 8.715544438707608,
 'te': 7.687817658198937,
 'qb': 16.003237164026448}

In [50]:
tracker = {}
board_control = board.copy()
total_rounds = 16
teams = 12

position = int(input("Position in Draft"))
outer = teams+1 - position
position_next = position
cur_pos = position_next

vl_wr, vl_qb, vl_rb, vl_te = 0,0,0,0 

for r in range(1, total_rounds+1, 1):
    prob_round = []
    prob_next_round = []
    prob_third_round = []
    print(cur_pos)
    display(pd.DataFrame(tracker, index=["Player", "Position", "VOR Loss when Selected", "WR Accumulated VOR Loss", "RB Accumulated VOR Loss", "QB Accumulated VOR Loss", "TE Accumulated VOR Loss"]))
    
    for i, player in board.iterrows():
        
        xppg = player["FPTS_TG_pred"]
        adp = player["AVG"]
        std = player["STD"]
                
        if r%2 == 0:
            position_next, position_third = (teams*(r))+position, (teams*(r+1))+outer
        else:
            position_next, position_third =  (teams*(r))+outer, (teams*(r+1))+position
            
        if std == 0:
            std = board[board.AVG < adp].STD.tail(5).mean()
            
        pr = round(1 - scipy.stats.norm(adp, std).cdf(cur_pos), 3)
        pr_next = round(1 - scipy.stats.norm(adp, std).cdf(position_next), 3)
        pr_third = round(1 - scipy.stats.norm(adp, std).cdf(position_third), 3)
        
        prob_round.append(pr)
        prob_next_round.append(pr_next)
        prob_third_round.append(pr_third)
    
    
    #position = position_next
    
    cur_pos = position_next
    
    board_control["prob_round"] = prob_round
    board_control["prob_next_round"] = prob_next_round
    board_control["prob_third_round"] = prob_third_round
    
    bc_sorted = board_control.sort_values("VOR", ascending=False)
    
    vor_loss_10 = []
    vor_loss_30 = []
    vor_loss_50 = []
    vor_loss_third = []
    
    for i, player in board.iterrows():
        player_pos = player["Position"]
        vor = player["VOR"]
        
        vor_loss_10.append(vor - bc_sorted[(bc_sorted.Position == player_pos) & (bc_sorted.prob_next_round > 0.1)].head(3)["VOR"].mean(numeric_only=True))
        vor_loss_30.append(vor - bc_sorted[(bc_sorted.Position == player_pos) & (bc_sorted.prob_next_round > 0.3)].head(3)["VOR"].mean(numeric_only=True))
        vor_loss_50.append(vor - bc_sorted[(bc_sorted.Position == player_pos) & (bc_sorted.prob_next_round > 0.5)].head(3)["VOR"].mean(numeric_only=True))
        #vor_loss_third.append(vor - bc_sorted[(bc_sorted.Position == player_pos) & (bc_sorted.prob_third_round > 0.5)].head(3)["VOR"].mean(numeric_only=True))
        
    #board_control["VOR Loss 10"] = vor_loss_10
    #board_control["VOR Loss 30"] = vor_loss_30
    board_control["VOR Loss 50"] = vor_loss_50
    #board_control["VOR Loss Third"] = vor_loss_third
    
    #board_control["Priority"] = board_control["VOR Loss 50"]*board_control["prob_round"]
    print("Top Players to Pick: ")
    top_table = board_control[board_control["prob_round"] >0.3].sort_values("VOR Loss 50", ascending=False)
    display(top_table.head(10))
    
    print("Round Players ADP")
    display(board_control[(board_control.AVG >=(r-1)*teams) & (board_control.AVG<=r*teams)])
    clear_output(wait=True)
    sel = int(input("Selected Player: "))
    pos_sel = board_control[board_control.pid == sel]["Position"].values[0]
    
    vl_wr += top_table[top_table.Position == "wr"]["VOR Loss 50"].values[0]
    vl_qb += top_table[top_table.Position == "qb"]["VOR Loss 50"].values[0]
    vl_rb += top_table[top_table.Position == "rb"]["VOR Loss 50"].values[0]
    vl_te += top_table[top_table.Position == "te"]["VOR Loss 50"].values[0]
    
    sel_vor = board_control[board_control.pid == sel].values[0][-1]
    
    if pos_sel == "wr":
        vl_wr-=sel_vor
    elif pos_sel == "rb":
        vl_rb-=sel_vor
    elif pos_sel == "qb":
        vl_qb-=sel_vor
    elif pos_sel == "te":
        vl_te-=sel_vor
    
    tracker["Round "+str(r)] = (board[board.pid == sel].values[0][1], pos_sel, sel_vor, vl_wr, vl_rb, vl_qb, vl_te)
    #if r == 10:
        #print("stop")
    #    break        
    

Selected Player: 19810


$ VOR Loss = VOR_p - VOR_l $

$ VOR_p = Player's\ VOR $

$ VOR_l = \sum{VOR}_{i=PP}^{PP+3} $

$ PP = Position\ of\ First\ Player\ with\ more\ than\ 0.5\ Prob\_Next\_Round$